In [20]:
#importing packages
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
import random
import os
from os import listdir
from PIL import Image
from sklearn.preprocessing import label_binarize,  LabelBinarizer
from keras.preprocessing import image
from tensorflow.keras.utils import array_to_img
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dropout, Dense
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
from keras.utils import to_categorical 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, LabelEncoder
warnings.filterwarnings("ignore")

In [7]:
color_data = pd.read_csv('../observations/color_feature.csv')
cta_pos = pd.read_csv('../observations/cta_position.csv')
eng_pos = pd.read_csv('../observations/engagement_position.csv')
logo_pos = pd.read_csv('../observations/logo_position.csv')
aa


In [8]:
color_data.head()

,c_code,rgb,occurrence,Assets,file_name
0,#000000,"(0, 0, 0)",6838,bfbf7a2b7ac635e67877b1ab87fd6629,endtext.png
1,#F9FFFF,"(249, 255, 255)",61593,bfbf7a2b7ac635e67877b1ab87fd6629,landing_2.jpg
2,#C4D8D9,"(196, 216, 217)",40416,bfbf7a2b7ac635e67877b1ab87fd6629,landing_2.jpg
3,#615B5D,"(97, 91, 93)",24738,bfbf7a2b7ac635e67877b1ab87fd6629,landing_2.jpg
4,#232122,"(35, 33, 34)",18858,bfbf7a2b7ac635e67877b1ab87fd6629,landing_2.jpg


In [9]:
cta_pos.head()

,Assets,located_image,base_image,top_left_X,top_left_Y,bottom_right_X,top_right_Y,height,width,total_image_height,total_image_width
0,bfbf7a2b7ac635e67877b1ab87fd6629,cta.png,_preview.png,104.0,215.0,404.0,279.0,64.0,300.0,900.0,600.0
1,945010afbf9a55bbdffcb0895f946155,cta.png,_preview.png,104.0,215.0,404.0,279.0,64.0,300.0,900.0,600.0
2,e68e20f592457b875ce29757ab855dfe,cta.png,_preview.png,366.0,154.0,600.0,204.0,50.0,234.0,500.0,600.0
3,adunit-nestle-purina-friskies-mob,cta,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,adunit-lionsgate-uwomt-user-slider-sensory-vid...,cta,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
eng_pos.head()

,Assets,located_image,base_image,top_left_X,top_left_Y,bottom_right_X,top_right_Y,height,width,total_image_height,total_image_width
0,bfbf7a2b7ac635e67877b1ab87fd6629,engagement_instruction.png,_preview.png,60.0,185.0,526.0,262.0,77.0,466.0,900.0,600.0
1,945010afbf9a55bbdffcb0895f946155,engagement_instruction.png,_preview.png,60.0,185.0,526.0,262.0,77.0,466.0,900.0,600.0
2,e68e20f592457b875ce29757ab855dfe,engagement_instruction.png,_preview.png,50.0,114.0,430.0,177.0,63.0,380.0,500.0,600.0
3,adunit-nestle-purina-friskies-mob,engagement,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,adunit-lionsgate-uwomt-user-slider-sensory-vid...,engagement,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
logo_pos.head()

,Assets,located_image,base_image,top_left_X,top_left_Y,bottom_right_X,top_right_Y,height,width,total_image_height,total_image_width
0,bfbf7a2b7ac635e67877b1ab87fd6629,logo,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,945010afbf9a55bbdffcb0895f946155,logo,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e68e20f592457b875ce29757ab855dfe,logo,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,adunit-nestle-purina-friskies-mob,logo,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,adunit-lionsgate-uwomt-user-slider-sensory-vid...,logo,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
color_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67954 entries, 0 to 67953
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   c_code      67954 non-null  object
 1   rgb         67954 non-null  object
 2   occurrence  67954 non-null  int64 
 3   Assets      67954 non-null  object
 4   file_name   67954 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.6+ MB


In [13]:
cta_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Assets              905 non-null    object 
 1   located_image       905 non-null    object 
 2   base_image          905 non-null    object 
 3   top_left_X          502 non-null    float64
 4   top_left_Y          502 non-null    float64
 5   bottom_right_X      502 non-null    float64
 6   top_right_Y         502 non-null    float64
 7   height              502 non-null    float64
 8   width               502 non-null    float64
 9   total_image_height  502 non-null    float64
 10  total_image_width   502 non-null    float64
dtypes: float64(8), object(3)
memory usage: 77.9+ KB


In [14]:
eng_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Assets              905 non-null    object 
 1   located_image       905 non-null    object 
 2   base_image          905 non-null    object 
 3   top_left_X          481 non-null    float64
 4   top_left_Y          481 non-null    float64
 5   bottom_right_X      481 non-null    float64
 6   top_right_Y         481 non-null    float64
 7   height              481 non-null    float64
 8   width               481 non-null    float64
 9   total_image_height  481 non-null    float64
 10  total_image_width   481 non-null    float64
dtypes: float64(8), object(3)
memory usage: 77.9+ KB


In [15]:
logo_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Assets              905 non-null    object 
 1   located_image       905 non-null    object 
 2   base_image          905 non-null    object 
 3   top_left_X          195 non-null    float64
 4   top_left_Y          195 non-null    float64
 5   bottom_right_X      195 non-null    float64
 6   top_right_Y         195 non-null    float64
 7   height              195 non-null    float64
 8   width               195 non-null    float64
 9   total_image_height  195 non-null    float64
 10  total_image_width   195 non-null    float64
dtypes: float64(8), object(3)
memory usage: 77.9+ KB


In [22]:
logo_pos.head(100)

,Assets,located_image,base_image,top_left_X,top_left_Y,bottom_right_X,top_right_Y,height,width,total_image_height,total_image_width
0,bfbf7a2b7ac635e67877b1ab87fd6629,logo,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,945010afbf9a55bbdffcb0895f946155,logo,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e68e20f592457b875ce29757ab855dfe,logo,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,adunit-nestle-purina-friskies-mob,logo,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,adunit-lionsgate-uwomt-user-slider-sensory-vid...,logo,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,015f38df736f5a9498b18a1d12170187,logo,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,db69df88f5a7b1540d4ff829bffb97f9,logo.jpg,_preview.png,0.0,0.0,600.0,900.0,900.0,600.0,900.0,600.0
97,22c8ca74c84370214de628417aff20d6,logo,_preview.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,d6875bfe73a3abc7eea25b7de121c643,logo.jpg,_preview.png,0.0,0.0,600.0,132.0,132.0,600.0,900.0,600.0


In [23]:
print('color',color_data.columns)
print('cta',cta_pos.columns)
print('eng',eng_pos.columns)
print('logo',logo_pos.columns)

color Index(['c_code', 'rgb', 'occurrence', 'Assets', 'file_name'], dtype='object')
cta Index(['Assets', 'located_image', 'base_image', 'top_left_X', 'top_left_Y',
       'bottom_right_X', 'top_right_Y', 'height', 'width',
       'total_image_height', 'total_image_width'],
      dtype='object')
eng Index(['Assets', 'located_image', 'base_image', 'top_left_X', 'top_left_Y',
       'bottom_right_X', 'top_right_Y', 'height', 'width',
       'total_image_height', 'total_image_width'],
      dtype='object')
logo Index(['Assets', 'located_image', 'base_image', 'top_left_X', 'top_left_Y',
       'bottom_right_X', 'top_right_Y', 'height', 'width',
       'total_image_height', 'total_image_width'],
      dtype='object')


In [24]:
merged_df = pd.merge(pd.merge(color_data,cta_pos,on='Assets'),eng_pos,on='Assets')

In [26]:
merged_df.head()

,c_code,rgb,occurrence,Assets,file_name,located_image_x,base_image_x,top_left_X_x,top_left_Y_x,bottom_right_X_x,...,located_image_y,base_image_y,top_left_X_y,top_left_Y_y,bottom_right_X_y,top_right_Y_y,height_y,width_y,total_image_height_y,total_image_width_y
0,#000000,"(0, 0, 0)",6838,bfbf7a2b7ac635e67877b1ab87fd6629,endtext.png,cta.png,_preview.png,104.0,215.0,404.0,...,engagement_instruction.png,_preview.png,60.0,185.0,526.0,262.0,77.0,466.0,900.0,600.0
1,#F9FFFF,"(249, 255, 255)",61593,bfbf7a2b7ac635e67877b1ab87fd6629,landing_2.jpg,cta.png,_preview.png,104.0,215.0,404.0,...,engagement_instruction.png,_preview.png,60.0,185.0,526.0,262.0,77.0,466.0,900.0,600.0
2,#C4D8D9,"(196, 216, 217)",40416,bfbf7a2b7ac635e67877b1ab87fd6629,landing_2.jpg,cta.png,_preview.png,104.0,215.0,404.0,...,engagement_instruction.png,_preview.png,60.0,185.0,526.0,262.0,77.0,466.0,900.0,600.0
3,#615B5D,"(97, 91, 93)",24738,bfbf7a2b7ac635e67877b1ab87fd6629,landing_2.jpg,cta.png,_preview.png,104.0,215.0,404.0,...,engagement_instruction.png,_preview.png,60.0,185.0,526.0,262.0,77.0,466.0,900.0,600.0
4,#232122,"(35, 33, 34)",18858,bfbf7a2b7ac635e67877b1ab87fd6629,landing_2.jpg,cta.png,_preview.png,104.0,215.0,404.0,...,engagement_instruction.png,_preview.png,60.0,185.0,526.0,262.0,77.0,466.0,900.0,600.0


In [27]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67954 entries, 0 to 67953
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   c_code                67954 non-null  object 
 1   rgb                   67954 non-null  object 
 2   occurrence            67954 non-null  int64  
 3   Assets                67954 non-null  object 
 4   file_name             67954 non-null  object 
 5   located_image_x       67954 non-null  object 
 6   base_image_x          67954 non-null  object 
 7   top_left_X_x          22611 non-null  float64
 8   top_left_Y_x          22611 non-null  float64
 9   bottom_right_X_x      22611 non-null  float64
 10  top_right_Y_x         22611 non-null  float64
 11  height_x              22611 non-null  float64
 12  width_x               22611 non-null  float64
 13  total_image_height_x  22611 non-null  float64
 14  total_image_width_x   22611 non-null  float64
 15  located_image_y    

In [28]:
features_df = pd.merge(merged_df,logo_pos,on='Assets')

In [29]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67954 entries, 0 to 67953
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   c_code                67954 non-null  object 
 1   rgb                   67954 non-null  object 
 2   occurrence            67954 non-null  int64  
 3   Assets                67954 non-null  object 
 4   file_name             67954 non-null  object 
 5   located_image_x       67954 non-null  object 
 6   base_image_x          67954 non-null  object 
 7   top_left_X_x          22611 non-null  float64
 8   top_left_Y_x          22611 non-null  float64
 9   bottom_right_X_x      22611 non-null  float64
 10  top_right_Y_x         22611 non-null  float64
 11  height_x              22611 non-null  float64
 12  width_x               22611 non-null  float64
 13  total_image_height_x  22611 non-null  float64
 14  total_image_width_x   22611 non-null  float64
 15  located_image_y    

In [30]:
features_df.isnull().sum()

c_code                      0
rgb                         0
occurrence                  0
Assets                      0
file_name                   0
located_image_x             0
base_image_x                0
top_left_X_x            45343
top_left_Y_x            45343
bottom_right_X_x        45343
top_right_Y_x           45343
height_x                45343
width_x                 45343
total_image_height_x    45343
total_image_width_x     45343
located_image_y             0
base_image_y                0
top_left_X_y            51397
top_left_Y_y            51397
bottom_right_X_y        51397
top_right_Y_y           51397
height_y                51397
width_y                 51397
total_image_height_y    51397
total_image_width_y     51397
located_image               0
base_image                  0
top_left_X              57489
top_left_Y              57489
bottom_right_X          57489
top_right_Y             57489
height                  57489
width                   57489
total_imag

In [31]:
features_df.shape

(67954, 35)

In [32]:
features_df.columns

Index(['c_code', 'rgb', 'occurrence', 'Assets', 'file_name', 'located_image_x',
       'base_image_x', 'top_left_X_x', 'top_left_Y_x', 'bottom_right_X_x',
       'top_right_Y_x', 'height_x', 'width_x', 'total_image_height_x',
       'total_image_width_x', 'located_image_y', 'base_image_y',
       'top_left_X_y', 'top_left_Y_y', 'bottom_right_X_y', 'top_right_Y_y',
       'height_y', 'width_y', 'total_image_height_y', 'total_image_width_y',
       'located_image', 'base_image', 'top_left_X', 'top_left_Y',
       'bottom_right_X', 'top_right_Y', 'height', 'width',
       'total_image_height', 'total_image_width'],
      dtype='object')